# Les réseaux convolutionnels

Réseau inspiré de http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf

## 1. Importation des librairies

In [1]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np

## 2. Chargement et conditionnement des données d'apprentissage

Chargement et normalisation des données

In [6]:

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)




La convention pour les images en entrée dans les réseaux de neurones convolutifs en PyTorch est d'utiliser des tenseurs de taille `(batch_size, channels, height, width)`. 

- `batch_size` correspond au nombre d'images dans un batch
- `channels` correspond au nombre de canaux de couleur de l'image. Pour une image en niveaux de gris, `channels` est égal à 1. Pour une image en couleur, `channels` est égal à 3 (rouge, vert, bleu)
- `height` correspond à la hauteur de l'image
- `width` correspond à la largeur 

## 3. Architecture du réseau 

Concevoir un réseau inspiré LeNet5 http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf
![LeNet5](https://miro.medium.com/max/2000/1*1TI1aGBZ4dybR6__DI9dzA.png)


La structure globale sera conservée (nombre de couches, nombre de neurones par couche…) mais dans cette version :
- Les couches de convolution sont totalement connectées
- La sortie est une couche fully connected (FC) classique 
- La fonction de coût est l’entropie croisée


Vous utiliserez la `nn.Sequential` pour le backbone et vous ajouterez une dernière couche pour la partie classlification

In [12]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, 1)
        self.conv2 = nn.Conv2d(6, 16, 5, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.gc = nn.Linear(84, 10)
        self.soft_max = nn.Softmax()

        self.relu = nn.ReLU()
        self.Avg_Pool2d = nn.AvgPool2d(2)
        

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.Avg_Pool2d(x)
        x = self.relu(self.conv2(x))
        x = self.Avg_Pool2d(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.gc(x)
        return self.soft_max(x, dim=1)

mnist_model = MNISTModel()


print(mnist_model)



MNISTModel(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (gc): Linear(in_features=84, out_features=10, bias=True)
  (soft_max): Softmax(dim=None)
  (relu): ReLU()
  (Avg_Pool2d): AvgPool2d(kernel_size=2, stride=2, padding=0)
)


## 4. Apprentissage du réseau

Définition des paramètres de l'optimisation (fonction de coût, méthode d'optimisation, hyperparamètres)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mnist_model.parameters(), lr=0.05)


train_loader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Boucle d'apprentissage (faire 10 epochs)

In [14]:
num_epochs = 10
losses = []
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = mnist_model(data)
        # converti automatiquement les labels dans targets en vecteur one hot
        loss = criterion(outputs, targets)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    
    #loss calculé uniquement sur le dernier batch de l'epoch
    print('epoch {}: {} (last batch)'.format(epoch, loss.item()))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4096x4 and 400x120)

## 5. Evaluation du modèle sur la base de test

On visualise le resultat pour `b_size` images et on les compare à la vérité terrain 

In [ ]:
#data, target = next(iter(test_loader))
b_size = 5
data = test_loader.dataset.data[0:b_size].data.unsqueeze(1).float()
target = test_loader.dataset.targets[0:b_size]
pred = mnist_model(data)
print(pred.argmax(dim=1, keepdim=False))
print(target)

On évalue les performances sur l'ensemble des données d'apprentissage

In [ ]:
test_loss = 0
correct = 0

# pour ne aps calculer les gradients (gain de temps et de mémoire)
with torch.no_grad():
    for data, target in test_loader:
        output = mnist_model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

## 6. Visualisation des filtres appris par le réseau

#### Ecrire une fonction de visualisation des filtres avec la signature suivante :  
`displayConvFilers(model, layer_name, num_filter=4, filter_size=(3,3), num_channel=0, fig_size=(2,2))`
- `model` : le réseau de neurones dont on souhaite visualiser les filtres
- `layer_name` : le nom de la couche dont on souhaite visualiser les filtres 
- `num_channel` : le numéro du canal dont on souhaite visualiser les filtres
- `fig_size` : les dimensions de la figure. Doivent être en cohérence avec le nombre de filtres de la couche

In [ ]:
#afficher la liste des noms des couches
filters = mnist_model.state_dict()
print(filters.keys())

#Pour récupérer les poids de la première couche
filters  = mnist_model.state_dict()['lenet5_backbone.0.weight']
filters.shape


In [ ]:
#Ces noms peuvent se déduire de l'affichage de la structure du réseau
print(mnist_model)

In [ ]:
def displayConvFilters(model, layer_name, num_channel=0, fig_size=(2,2)):
    ...


Appeler deux fois la fonction pour afficher les filtres de la première couche et le premier canal des filtres de la deuxième couche de convolution

In [ ]:
...